# Rolling Planning mit Tardiness (Max)

In [ ]:
from configs.config import get_path
import src.utils.converter as convert
import src.utils.presenter as show
import src.utils.checker as check

import pandas as pd
pd.set_option('display.max_rows', 21)

import src.utils.rolling_scheduling as rolling_schedule

import src.models.lp.tardiness_solver as solver
from src.simulation.ProductionDaySimulation import ProductionDaySimulation
import time

In [ ]:
max_time = 60*60*3 # 3h

## Laden der Initialisierungsdaten

In [ ]:
data_path = get_path("data", "lp_tardiness")

df_jssp = pd.read_csv(data_path / "02_tardiness_all_jssp.csv")
df_times = pd.read_csv(data_path / "02_tardiness_all_times.csv")

### Tag 0 

In [ ]:
df_plan = pd.read_csv(data_path / "02_tardiness_init_plan.csv")
df_execution = pd.read_csv(data_path / "02_tardiness_execution.csv")
df_undone = pd.read_csv(data_path / "02_tardiness_init_undone.csv")

### Settings

In [ ]:
def get_schedule_filename(prefix: str = "", day: int = 0, data_txt: str = "schedule", suffix: str = "") -> str:
    file_template = "{prefix}_{data}_{day:02d}{suffix}.csv"
    if suffix:
        suffix = f"_{suffix}"
    return data_path / file_template.format(prefix=prefix,data=data_txt,day=day, suffix=suffix)

## Params

In [ ]:
# fix
day_length = 1440
horizon_days = 2

notebook_prefix = "3b"

#### Rescheduling für Tage 1-5
- Tag 1: Ankunfttermine in [1, 2] + unabgeschlosse von Vortagen (evtl. Tag 0)
- Tag 2: Ankunfttermine in [2, 3] + unabgeschlosse von Vortagen (evtl. Tag 0 "oder" Tag 1)
- ...
- Tag 5: Ankunfttermine in [5, 6] + unabgeschlosse von Vortagen

In [ ]:
first_start = 1
last_planning_start = 5

In [ ]:
for day_numb in range(first_start, last_planning_start + 1):
    day_start = day_length * day_numb
    day_end = day_start + day_length
    planning_end = day_start + horizon_days * day_length

    # ------------------- I. Ankunfts- und Operationsvorbereitung -------------------
    df_jssp_curr, df_times_curr = rolling_schedule.filter_jobs_by_arrival_window(df_times, df_jssp, day_start, planning_end)
    df_jssp_curr = rolling_schedule.extend_with_undone_operations(df_jssp_curr, df_undone)
    df_times_curr = rolling_schedule.update_times_after_operation_changes(df_times, df_jssp_curr)

    # ------------------- II. Relevante laufende Operationen -------------------------
    df_execution_important = rolling_schedule.get_operations_running_into_day(df_execution, day_start)

    # ------------------- III. Rescheduling durchführen -------------------------------

    starting_time = time.time()
    df_plan = solver.solve_jssp_max_tardiness_with_devpen(df_jssp_curr, df_times_curr, df_execution_important, 
                                                        df_original_plan = df_plan, # prev. Plan
                                                        r = 0.45,                   # 45% Tardiness, 55% DevPen
                                                        reschedule_start = day_start,
                                                        solver= "HiGHS", msg=False, timeLimit=max_time, gapRel= 0.05, threads=7)
    solver_duration = time.time() - starting_time
    print(f"\n  Scheduling-Dauer: {int(solver_duration // 60)} Minuten und {(solver_duration % 60):.2f} Sekunden.")

    df_plan.to_csv(get_schedule_filename(notebook_prefix, day=day_numb), index=False)

    show.plot_gantt_machines(df_plan, title=f"Gantt-Diagramm ab Tag {day_numb}")
    check.check_constraints(df_plan)

    # ------------------- IV. Einen Tag simulieren -------------------------------------

    simulation = ProductionDaySimulation(df_plan, vc=0.25)
    df_execution, df_undone = simulation.run(start_time=day_start, end_time=day_end)
    if not df_execution.empty:
        show.plot_gantt_machines(df_execution, title=f"Gantt-Diagramm für Simulationstag {day_numb}", duration_column="Simulated Processing Time")
    else:
        print(f"Nothing executed on day {day_numb}")